In [1]:
import os
import sys
from pathlib import Path

In [34]:
project_name = 'scosy'
# notebook meant to be under ../scosy/notebookts/pubmed.ipynb
project_path = Path(os.getcwd()).parent
template_path = Path(project_path, 'template')
similarities_path = Path(data_path, 'similarities')
data_path = Path(project_path, 'dataset')

# including the project folder and the utils folder
if str(project_path) not in ''.join(sys.path):
    sys.path.extend([str(project_path)])

print('project path = {0}'.format(project_path))
print('data path = {0}'.format(data_path))
print('sys.path = {0}'.format(sys.path))

project path = /home/guerramarj/github/scosy
data path = /home/guerramarj/github/scosy/dataset
model path = /home/guerramarj/github/scosy/models
embedding path = /home/guerramarj/github/scosy/embedding
sys.path = ['', '/cm/shared/easybuild/software/EasyBuild/3.8.1/lib/python2.7/site-packages/vsc_install-0.11.3-py2.7.egg', '/cm/shared/easybuild/software/EasyBuild/3.8.1/lib/python2.7/site-packages/vsc_base-2.8.3-py2.7.egg', '/cm/shared/easybuild/software/EasyBuild/3.8.1/lib/python2.7/site-packages/easybuild_framework-3.8.1-py2.7.egg', '/cm/shared/easybuild/software/EasyBuild/3.8.1/lib/python2.7/site-packages/easybuild_easyblocks-3.8.1-py2.7.egg', '/cm/shared/easybuild/software/EasyBuild/3.8.1/lib/python2.7/site-packages/easybuild_easyconfigs-3.8.1-py2.7.egg', '/cm/shared/easybuild/software/EasyBuild/3.8.1/lib/python2.7/site-packages', '/home/guerramarj/packages/anaconda3/envs/nlp/lib/python36.zip', '/home/guerramarj/packages/anaconda3/envs/nlp/lib/python3.6', '/home/guerramarj/packages/a

In [58]:
# utils
import argparse
import pandas as pd
import numpy as np
import csv
# pubmed
from utils.parse import parse
from Bio import Entrez
# processing mesh treee hierarchy
import re
# flatten list
from functools import reduce
from operator import iconcat
# nlp
from nltk.tokenize import word_tokenize 
import gensim

In [4]:
filename = Path(template_path, '2019New_Mesh_Tree_Hierarchy.txt')
mesh_description_dict = dict()
with filename.open('r') as tree_file:
    header = ''
    for line_ix, line in enumerate(tree_file):
            if line_ix == 0:
                header = line
            # '\n' -> empty line
            # 'TREE_NUMBER    DESCRIPTOR' -> header line
            # '-------------------------' -> visual line
            if line != '\n' and line != header and '----' not in line:
                # subtitute 2 or more spaces by a comma and
                # remove last comma if it appears
                line = re.sub('[\s]{2,}', ',', line).rstrip(',')
                tree_num_desc = line.split(',')
                # this is done for keys with len > 50 (special cases)
                value = tree_num_desc[0]
                key = ' '.join(tree_num_desc[1:])
                key.replace('  ', '')
                mesh_description_dict[key.lower()] = value

In [47]:
data_file = Path(data_path, 'results.xml')
records_handle = data_file.open()
fetch_records = parse(handle=records_handle)

In [48]:
# contains all the metadata elements on the paper level: PubMed unique Identifier number(PMID), Title, Abstract,
# Year, Month, AuthorList, SubjectList, date
paper_df = pd.DataFrame(columns=['pmid', 'title', 'abstract', 'mesh'])

for record in fetch_records:

    pmid = record.get('PMID')
    title = record.get('TI')
    abstract = record.get('AB')
    mesh_term = record.get('MH')
    
    if pmid and abstract:
        
        if mesh_term and len(mesh_term) > 1:
            # divide all the mesh with multiple term
            mesh_term = [x.replace('*', '').lower().split('/')for x in mesh_term]
            # flatten the list
            mesh_term = reduce(iconcat, mesh_term, [])

        row = pd.DataFrame([[pmid, title, abstract, mesh_term]],
                           columns=['pmid', 'title', 'abstract', 'mesh'])
        paper_df = paper_df.append(row, ignore_index=True)

In [50]:
paper_df.to_csv(Path(data_path, 'pmid_title_abstract_mesh.csv'),index=False)

In [51]:
paper_df.head()

,pmid,title,abstract,mesh
0,30500988,Early Post-Intensive Care Syndrome among Older...,BACKGROUND/OBJECTIVES: New or worsened disabil...,None
1,30500532,Revitalizing Aging Skin through Diet.,Mechanisms underlying aging of the skin dermis...,None
2,30499797,A Partially Structured Postoperative Handoff P...,OBJECTIVE: To assess the effectiveness of stan...,None
3,30499597,Automated data extraction and ensemble methods...,PURPOSE: To compare the effectiveness of ensem...,None
4,30499112,Modeling Epidermal Growth Factor Inhibitor-Med...,We have demonstrated that lung sparing surgery...,None


In [63]:
text = paper_df['title'].astype(str) + paper_df['abstract'].astype(str) + paper_df['mesh'].astype(str)
text = [x.replace('None', '') for x in text]
text = pd.DataFrame({'pmid': paper_df.pmid, 'text':text})

In [65]:
text.to_csv(Path(data_path, 'pmid_text.csv'), index=False)

In [8]:
raw_documents = pd.read_csv(Path(data_path, 'pmid_text.csv'))
raw_documents.head()

,pmid,text
0,30500988,Early Post-Intensive Care Syndrome among Older...
1,30500532,Revitalizing Aging Skin through Diet.Mechanism...
2,30499797,A Partially Structured Postoperative Handoff P...
3,30499597,Automated data extraction and ensemble methods...
4,30499112,Modeling Epidermal Growth Factor Inhibitor-Med...


In [10]:
gen_docs = [[w.lower() for w in word_tokenize(text)] 
            for text in raw_documents.text]
print(gen_docs[:5])

[['early', 'post-intensive', 'care', 'syndrome', 'among', 'older', 'adult', 'sepsis', 'survivors', 'receiving', 'home', 'care.background/objectives', ':', 'new', 'or', 'worsened', 'disabilities', 'in', 'functional', ',', 'cognitive', ',', 'or', 'mental', 'health', 'following', 'an', 'intensive', 'care', 'unit', '(', 'icu', ')', 'stay', 'are', 'referred', 'to', 'as', 'post-intensive', 'care', 'syndrome', '(', 'pics', ')', '.', 'pics', 'has', 'not', 'been', 'described', 'in', 'older', 'adults', 'receiving', 'home', 'care', '.', 'our', 'aim', 'was', 'to', 'examine', 'the', 'relationship', 'between', 'length', 'of', 'icu', 'stay', 'and', 'pics', 'among', 'older', 'adults', 'receiving', 'home', 'care', '.', 'we', 'expected', 'that', 'patients', 'in', 'the', 'icu', 'for', '3', 'days', 'or', 'longer', 'would', 'demonstrate', 'significantly', 'more', 'disability', 'in', 'all', 'three', 'domains', 'on', 'follow-up', 'than', 'those', 'not', 'in', 'the', 'icu', '.', 'a', 'secondary', 'aim', 'was'

In [21]:
# create a dictionary to match tokens to integers
dictionary = gensim.corpora.Dictionary(gen_docs)
print(dictionary[5])
print(dictionary.token2id['road'])
print("Number of words in dictionary:",len(dictionary))

1
6152
Number of words in dictionary: 82783


In [22]:
# lists the number of times each word occurs in the document
# list of tuples
    # first = index of the word
    # second = number of time that appears in that document
corpus = [dictionary.doc2bow(gen_doc) for gen_doc in gen_docs]

In [31]:
# number of unique tokens in the first document
len(corpus[0])

175

<pre>
term frequency inverse term frequency (tf-idf):
    term frequency = how often a word shows up in a document
    inverse document frequency = scale that value by how rare the word is in the corpus
    

In [33]:
tf_idf = gensim.models.TfidfModel(corpus)
print(tf_idf)
s = 0
for i in corpus:
    s += len(i)
print(s)
# num_nnz = number of tokens

TfidfModel(num_docs=8367, num_nnz=1237181)
1237181


In [52]:
index = gensim.similarities.Similarity(str(similarities_path),
                                       tf_idf[corpus], 
                                       num_features=len(dictionary)) 
query = next(iter(corpus))
result = index[query]  # search similar to `query` in index

In [66]:
# yield similarities of the indexed documents
with Path(similarities_path, 'document_0_50.txt').open('w+') as out_file:
    writer = csv.writer(out_file, delimiter=',')
    for doc_sim_ix, doc_sim in enumerate(index):
        if doc_sim_ix == 51:
            break
        pmid = raw_documents.loc[doc_sim_ix]['pmid']
        c_doc = 'document = {0}, pmid = {1}'.format(doc_sim_ix, pmid)
        print(c_doc)
        writer.writerow([pmid])
        writer.writerow(list(enumerate(sim_result)))
        out_file.flush()

document = 0, pmid = 30500988
document = 1, pmid = 30500532
